In [ ]:
# import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/DeepLearningProj

/content/drive/MyDrive/DeepLearningProj


In [ ]:
rom_dir = '/content/drive/MyDrive/DeepLearningProj/romantic_train.txt'
funny_dir = '/content/drive/MyDrive/DeepLearningProj/funny_train.txt'

In [ ]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# To remove punctuations
def remove_punctuation(text_original):
    text_no_punctuation = text_original.translate(string.punctuation)
    return(text_no_punctuation)


# To remove single characters
def remove_single_character(text):
    text_len_more_than1 = ""
    for word in text.split():
        if len(word) > 1:
            text_len_more_than1 += " " + word
    return(text_len_more_than1)

# To remove numeric values
def remove_numeric(text,printTF=False):
    text_no_numeric = ""
    for word in text.split():
        isalpha = word.isalpha()
        if printTF:
            print("    {:10} : {:}".format(word,isalpha))
        if isalpha:
            text_no_numeric += " " + word
    return(text_no_numeric)

In [ ]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'rb')
	# read all text
	text = file.read().decode(errors='replace')
	# close the file
	file.close()
	return text

In [ ]:
rom_text = load_doc(rom_dir)
funny_text = load_doc(funny_dir)
print(rom_text[:325])
print("==============")
print(funny_text[:325])


In [ ]:
lines = rom_text.split("\n")
print(len(lines))
# text_pairs = []
rom_sens=[]
for line in lines:
    rom= line.split("\n")
    # rom = "[start] " + rom + " [end]"
    # text_pairs.append((eng, spa))
    rom_sens.append(rom)

7000


In [ ]:
lines = funny_text.split("\n")[:-2]
print(len(lines))
# text_pairs = []
funny_sens=[]
for line in lines:
    funny= line.split("\n")
    # spa = "[start] " + spa + " [end]"
    # text_pairs.append((eng, spa))
    funny_sens.append(funny)

7000


In [ ]:
import spacy, en_core_web_sm
nlp = en_core_web_sm.load()
funny_par = []
doc=nlp("several children playing on a polaris vehicle enjoying the sun in the great outdoors")
#doc text is from spacy website


In [ ]:
for x in doc :
  s = ""
  if x.pos_ == "NOUN" or x.pos_ == "PROPN" or x.pos_=="PRON" or x.pos_ == "VERB" or x.pos == "NUM":
    s = s+str(x)
  funny_par.append(s)

In [ ]:
doc = nlp(funny_sens[0][0].split('\r')[0])
s = []
for x in doc :
  
  if x.pos_ == "NOUN" or x.pos_ == "PROPN" or x.pos_=="PRON" or x.pos_ == "VERB" or x.pos == "NUM":
    #print(x)
    s.append(str(x))
#s = s[1:]
print(s[:len(s)-2])

i = funny_sens[0][0].split('\r')[0].find(s[-3])
print(funny_sens[0][0].split('\r')[0][:i+len(s[-3])])

['dogs', 'chase', 'ground', 'search']
two dogs chase each other across the snowy ground in search


In [ ]:
funny_par = []
for fun in funny_sens:
  doc = nlp(fun[0].split('\r')[0])
  s = []
  w = ""
  for x in doc :
    if x.pos_ == "NOUN" or x.pos_ == "PROPN" or x.pos_ == "VERB" or x.pos == "NUM":
      s.append(str(x))
  try:
    i = fun[0].split('\r')[0].find(s[-3])
    funny_par.append(fun[0].split('\r')[0][:i+len(s[-3])] + ".")
  except:
    i = fun[0].split('\r')[0].find(s[-1])
    funny_par.append(fun[0].split('\r')[0][:i-1] + ".")



  

In [ ]:
def remove_punctuation(text_original):
    text_no_punctuation = text_original.translate(string.punctuation)
    return(text_no_punctuation)


# To remove single characters
def remove_single_character(text):
    text_len_more_than1 = ""
    for word in text.split():
        if len(word) > 1:
            text_len_more_than1 += " " + word
    return(text_len_more_than1)
    
def remove_punctuation(text_original):
    text_no_punctuation = text_original.translate(string.punctuation)
    return(text_no_punctuation)

# To remove single characters
def remove_single_character(text):
    text_len_more_than1 = ""
    for word in text.split():
        if len(word) > 1:
            text_len_more_than1 += " " + word
    return(text_len_more_than1)

def text_clean(text_original):
    text = remove_punctuation(text_original)
    text = remove_single_character(text)
    #text = remove_numeric(text)
    return(text)
    


In [ ]:
for i in range(len(funny_sens)):
    funny_sens[i] = text_clean(str(funny_sens[i]))

for i in range(len(rom_sens)):
    rom_sens[i] = text_clean(str(rom_sens[i]))

In [ ]:
text_pairs2=[]
for i in range(len(rom_sens)):
  funny_temp="[start] "+(funny_sens[i].split('\\r')[0])[3:]+" [end]"
  text_pairs2.append(((rom_sens[i].split('\\r')[0])[3:],funny_temp))

In [ ]:
text_pairs=[]
for i in range(len(funny_sens)):
  funny_temp="[start] "+funny_sens[i][0].split('\r')[0]+" [end]"
  text_pairs.append((funny_par[i],funny_temp))

In [ ]:
random.shuffle(text_pairs)
num_val_samples = int(0.1 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

7000 total pairs
5950 training pairs
1050 validation pairs
1050 test pairs


In [ ]:

strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


rom_vectorization = TextVectorization(
    max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length,
)
funny_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
)
train_rom_texts = [pair[0] for pair in train_pairs]
train_funny_texts = [pair[1] for pair in train_pairs]
#rom_vectorization.adapt(train_rom_texts)
#funny_vectorization.adapt(train_funny_texts)

text_rom_texts = [pair[0] for pair in text_pairs]
text_funny_texts = [pair[1] for pair in text_pairs]
text_combined = text_rom_texts + text_funny_texts
rom_vectorization.adapt(text_combined)
funny_vectorization.adapt(text_combined)


In [ ]:
voc_f = funny_vectorization.get_vocabulary()
word_index_f = dict(zip(voc_f, range(len(voc_f))))

voc_r = rom_vectorization.get_vocabulary()
word_index_r = dict(zip(voc_r, range(len(voc_r))))

In [ ]:

def format_dataset(rom, funny):
    rom = rom_vectorization(rom)
    funny = funny_vectorization(funny)
    return ({"encoder_inputs": rom, "decoder_inputs": funny[:, :-1],}, funny[:, 1:])


def make_dataset(pairs):
    rom_texts, funny_texts = zip(*pairs)
    rom_texts = list(rom_texts)
    funny_texts = list(funny_texts)
    dataset = tf.data.Dataset.from_tensor_slices((rom_texts, funny_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

Let's take a quick look at the sequence shapes
(we have batches of 64 pairs, and all sequences are 20 steps long):

In [ ]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


**Word Embedding**

In [ ]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip -q glove.6B.zip

In [ ]:
import os

path_to_glove_file = os.path.join(
    #os.path.expanduser("~"), ".keras/datasets/glove.6B.100d.txt"
    os.path.expanduser("~"), ".glove.6B.100d.txt"
)
path_to_glove_file = os.path.join("glove.6B.100d.txt")

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

In [ ]:
num_tokens = len(voc_f) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index_f.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
        
print("Converted %d words (%d misses)" % (hits, misses))

In [ ]:
from tensorflow.keras.layers import Embedding

embedding_layer_f = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

Romantic Vectorization

In [ ]:
num_tokens_r = len(voc_r) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens_r, embedding_dim))
for word, i in word_index_r.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
        
print("Converted %d words (%d misses)" % (hits, misses))

Converted 8407 words (607 misses)


In [ ]:
from tensorflow.keras.layers import Embedding

embedding_layer_r = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [ ]:

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)

        self.token_embeddings = Embedding(
        num_tokens,
        embedding_dim,
        embeddings_initializer=keras.initializers.Constant(embedding_matrix),
        trainable=False,
    )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)


In [ ]:
embed_dim = 100
latent_dim = 800
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")

#x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder_outputs = layers.Dropout(0.2)(encoder_outputs)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")

encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")


x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.2)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)


In [ ]:
epochs = 30

transformer.summary()
transformer.compile(
    tf.keras.optimizers.Adam(lr=0.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

p="./model_Transfer3.h5"
transformer.load_weights(p)
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=25, verbose=0, mode='max')

history=transformer.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=[earlyStopping])#,callbacks=[earlyStopping,mcp_save,reduce_lr_loss]


In [ ]:
funny_vocab = funny_vectorization.get_vocabulary()
funny_index_lookup = dict(zip(range(len(funny_vocab)), funny_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = rom_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = funny_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = funny_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_rom_texts = [pair[0] for pair in test_pairs]
for _ in range(30):
    input_sentence = random.choice(test_rom_texts)
    translated = decode_sequence(input_sentence)
    print(input_sentence)
    print(translated)